In [1]:
pip install tabula


  Preparing metadata (setup.py) ... done
Using legacy 'setup.py install' for tabula, since package 'wheel' is not installed.
  Running setup.py install for tabula ... done

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ipywidgets


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip3.9 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import tabula
import pandas as pd
import ipywidgets as widgets
import os

# Define the function for extracting tables from PDF files and writing them to CSV
def extract_tables(pdf_files, combine_sheets, output_format):
    # Create separate folders for CSV and XLSX files
    os.makedirs("CSV", exist_ok=True)
    os.makedirs("XLSX", exist_ok=True)

    # Create an empty list to store the extracted tables
    tables = []

    # Iterate through the PDF files
    for file in pdf_files:
        # Extract tables from each page of the PDF
        extracted_tables = tabula.read_pdf(file, pages="all", multiple_tables=True)
        tables.extend(extracted_tables)

    # Check if tables need to be combined in one sheet or separate sheets
    if combine_sheets:
        # Concatenate all tables into one DataFrame
        combined_data = pd.concat(tables, ignore_index=True)

        # Write the combined data to the output file
        if output_format == 'XLSX':
            combined_data.to_excel("XLSX/output.xlsx", index=False)
            print("Data extraction complete. Output file saved as: XLSX/output.xlsx")
        elif output_format == 'CSV':
            combined_data.to_csv("CSV/output.csv", index=False)
            print("Data extraction complete. Output file saved as: CSV/output.csv")
    else:
        # Write each table to a separate sheet in the Excel file or separate CSV files
        if output_format == 'XLSX':
            with pd.ExcelWriter("XLSX/output.xlsx") as writer:
                for i, table in enumerate(tables):
                    sheet_name = f"Sheet{i+1}"
                    table.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"Table {i+1} saved in Sheet {sheet_name} of XLSX/output.xlsx")

            print("Data extraction complete. Output file saved as: XLSX/output.xlsx")
        elif output_format == 'CSV':
            for i, table in enumerate(tables):
                file_name = f"CSV/output{i+1}.csv"
                table.to_csv(file_name, index=False)
                print(f"Table {i+1} saved as {file_name}")

            print("Data extraction complete. Output files saved in the CSV folder")

# Define the function for handling the button click event
def on_submit_button_click(b):
    # Get the selected PDF files
    pdf_files = []
    for file_picker in file_pickers:
        if file_picker.value:
            uploaded_file = next(iter(file_picker.value.values()))
            content = uploaded_file['content']
            with open(uploaded_file['metadata']['name'], 'wb') as f:
                f.write(content)
            pdf_files.append(uploaded_file['metadata']['name'])

    # Get the combine sheets option
    combine_sheets = combine_sheets_checkbox.value

    # Get the selected output format
    output_format = output_format_dropdown.value

    # Extract the tables and write them to the output file
    extract_tables(pdf_files, combine_sheets, output_format)

# Define the file pickers for selecting PDF files
file_pickers = [widgets.FileUpload(accept=".pdf") for _ in range(5)]  # Adjust the range according to your needs

# Define the checkbox for selecting whether to combine sheets or not
combine_sheets_checkbox = widgets.Checkbox(value=True, description='Combine tables in one sheet')

# Define the dropdown for selecting the output format
output_format_dropdown = widgets.Dropdown(options=['XLSX', 'CSV'], value='XLSX', description='Output Format:')

# Define the submit button
submit_button = widgets.Button(description="Submit")

# Register the button's click event
submit_button.on_click(on_submit_button_click)

# Display the file pickers, checkbox, dropdown, and submit button
display(widgets.VBox(file_pickers + [combine_sheets_checkbox, output_format_dropdown, submit_button]))

